# Terminal Link Optimization

This is the main ipython script for the terminal link optimization algorithm.
I will update the documentation accordingly once the project has progressed further.

## Import packages

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Make simulation of satellite and customers

Lets first start by simulating customers on the ground. This will essentially 
end up being a straight line with dots (representing customers) being randomly 
distributed along the line. The line will be spaced according to some 
arbitrary time metric. Each customer will have (for now) 4 attributes which 
describe the "importantness" of each customer.

In [2]:
"""
Lets make an array containing the physical location of some simulated customers.
The array will have the shape (N_time_segments,length_of_time_segment)

customer data array `customer_arr` will be of shape (number of training orbits, number of customers 
in orbit, number of features describing priority of customers)
"""

# training set hyperparameters
num_customers = 100 # total number of customers on planet
num_orbits = 10000  # total number of training samples (in this case orbits)
customer_arr = np.zeros((num_orbits,num_customers,5)) # Define emtpy array to contain customer data

for i in range(num_orbits):
    # Make customer index labels
    customer_arr[i,:,0] = np.arange(start=0,stop=num_customers,step=1)

    # Define random locations of customers on a line defining the planet
    customer_arr[i,:,1] = np.random.randint(low=0,high=10000,size=(num_customers))
    
    # We will Define 4 variables which describe the `importantness` of each customer

    # Assign a random customer important factor. 0 == low importance, 1 == high importance
    customer_arr[i,:,2] = np.random.uniform(low=0.0,high=1.0,size=(num_customers))

    # Assign a random customer weather factor. 0 == high cloud coverage, 1 == low cloud coverage
    customer_arr[i,:,3] = np.random.uniform(low=0.0,high=1.0,size=(num_customers))



## Choose optimal order in which to distribute keys (simple approach)

This is an incredibly simple algorithm which only takes into account 
the importantness and weather factors for each customer. It does not 
take into account the total time customers have been waiting for 
a key to be distributed to them.

In [3]:
# Loop over all available customers within line-of-sight (this is by default set to num_customers)
# This is an example using only one training orbit.
for i in range(num_customers):
    customer_arr[0,i,4] = customer_arr[0,i,2] * customer_arr[0,i,3]

customer_prob_list_idx = np.argsort(customer_arr[0,:,4])[::-1]
customer_prob_list = customer_arr[0,customer_prob_list_idx,0]

In [4]:
# Print out ordered list of customers to distribute keys to
customer_prob_list

array([23., 50., 24., 36., 11., 16., 19., 47., 53., 87., 67., 28., 99.,
       58., 71., 83., 21.,  2., 30., 74., 97., 79., 25., 91., 37.,  3.,
       64., 65., 45., 44., 40., 13., 42., 15., 98., 60., 55., 72., 35.,
        6., 14.,  5., 68., 32., 20., 75., 52., 57., 66., 29., 78., 81.,
       86., 17., 51.,  8., 41., 48., 46., 89., 33., 84., 90., 63., 31.,
       77., 88., 85., 80., 26.,  9., 62., 27., 73., 34., 43., 96., 10.,
       22., 39., 92.,  4.,  0.,  7., 70., 54., 95., 61., 38., 49., 12.,
       69., 94., 76.,  1., 93., 59., 82., 18., 56.])

## Neural Network Approach (complicated approach)

I've written some pseudo code down on a piece of paper. Not guranteeing that 
this will actually work in practice, but I will make my first attempt of this here.

### Define network archetecture

In [5]:
# This approach uses the Pytorch neural network library

# neural network hyperparameters
number_epochs = 100
batch_size = 64

# Define network archetecture (convolutional neural network)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3).double()
        self.conv2 = nn.Conv2d(32, 64, 3).double()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(1920, 120).double()
        self.fc2 = nn.Linear(120, 84).double()
        self.fc3 = nn.Linear(84, 1).double() # one node for predicted probability

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        # Two fully-connected hidden layers and one output layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.to(device)
print(net)

params = list(net.parameters())

criterion = nn.MSELoss()

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1920, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)


In [7]:
def custom_loss(output, target, batch_feature_data):
    """ This is a custom loss function which 
    takes as input the raw output from the neural 
    network and returns a loss which attempts to 
    minimize the number of keys not distributed 
    within the alloted amount of time.
    """
    
    def my_model(output, batch_feature_data):
        """ This is a model which will iterate 
        over the predicted ordering of keys to be 
        distributed. There will be a penalty 
        associated with any keys which are predicted to 
        be distributed outside of the obersvation window.
        """
    
        return num_pred_outside_window
    
    # Get number of keys predicted to be outside of target window
    num_pred_outside_window = my_model(output, batch_feature_data)
    
    loss = torch.mean((num_pred_outside_window - target)**2)
    return loss

### Run network over entire training set

In [8]:
# training features: location, importance, weather
X_train = customer_arr[:,:,1:4]
Y_train = np.zeros((num_orbits,1))

In [11]:
running_loss = 0.0
train_split = num_orbits

dset_train = TensorDataset(torch.tensor(X_train), torch.tensor(Y_train))
dataloader = DataLoader(dset_train, batch_size=batch_size,
                        shuffle=True)

# Iterate over entire training set
for epoch_num in range(number_epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        
        # in your training loop:
        if (train_split - (batch_size * i_batch)) < batch_size:
            break
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(sampled_batch[0].reshape(batch_size,1,num_customers,3).to(device))
        print('Made it through iteration!')
#        loss = custom_loss(output, Y_train.reshape(batch_size,1).to(device), sampled_batch)
#        loss.backward()
#        optimizer.step()    # Does the update
        
    
    
#    print('Training epoch %d/%d' % (epoch_num+1,number_epochs))
    # print statistics
#    running_loss += loss.item()
#    if i % int(X_train.shape[0]/batch_size) == (int(X_train.shape[0]/batch_size) - 1):    # print every 2000 mini-batches
#        print('loss: %.3f' %
#                (running_loss / 10))
#        running_loss = 0.0
#        epoch_num += 1
#        print('Training epoch %d/%d' % (epoch_num,number_epochs))

RuntimeError: CUDA out of memory. Tried to allocate 17179869184.00 GiB (GPU 0; 1.96 GiB total capacity; 7.17 MiB already allocated; 1.06 GiB free; 974.00 KiB cached)